In [2]:
import os
import numpy as np
import cv2
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import random
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda, Dropout, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
#import pandas as pd
from PIL import Image
import urllib
#import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import time

In [3]:
train_dataset = pd.read_csv('sign_data/train_data.csv')
test_dataset = pd.read_csv('sign_data/train_data.csv')

In [4]:
len(train_dataset)

23205

In [5]:
train_dir = "sign_data/train"
test_dir = "sign_data/test"

In [6]:
class DataLoader:
  #constructor
  def __init__(self, dataset, batch_size,dir):
    self.dataset = dataset
    self.batch_size = batch_size
    self.dir = dir
  #shuffler
  def shuffle(self):
    return self.dataset.sample(frac=1)
  #generator
  def datagen(self):
    num_samples = len(self.dataset)
    while True:
        # shuffling the samples
        self.dataset = self.shuffle()
        #iterates over the dataset , store img paths,extract labels and empty arrays image1, image2, labels
        for batch in range(1, num_samples, self.batch_size):
            #path
            image1_batch_samples = self.dir + "/" + self.dataset.iloc[:, 0][batch:batch + self.batch_size]
            image2_batch_samples = self.dir + "/" + self.dataset.iloc[:, 1][batch:batch + self.batch_size]
            label_batch_samples = self.dataset.iloc[:, 2][batch:batch + self.batch_size]
            Image1, Image2, Label = [], [], []
            for image1, image2, label in zip(image1_batch_samples, image2_batch_samples, label_batch_samples):
                # append them to Images directly
                image1_data = Image.open(image1)
                image2_data = Image.open(image2)
                # resizing the images
                image1_data = image1_data.resize((112, 112))
                image2_data = image2_data.resize((112, 112))
                # converting to array
                image1_data = img_to_array(image1_data)
                image2_data = img_to_array(image2_data)
                Image1.append(image1_data)
                Image2.append(image2_data)
                Label.append(label)
            # convert each list to numpy arrays to ensure that they get processed by fit function
            Image1 = np.asarray(Image1).astype(np.float32)
            Image2 = np.asarray(Image2).astype(np.float32)
            Label = np.asarray(Label).astype(np.float32)
            # yeild 1 at a time
            yield [Image1, Image2], Label


In [7]:
train_set, val_set = train_test_split(train_dataset, test_size=0.25)


In [8]:
train_gen= DataLoader(train_set,32,train_dir) #total batches 32
val_gen = DataLoader(val_set,32,train_dir)

In [9]:
train_batch = next(train_gen.datagen())
print("Train batch images shape:", train_batch[0][0].shape, train_batch[0][1].shape)
print("Train batch labels shape:", train_batch[1].shape)


Train batch images shape: (32, 112, 112, 3) (32, 112, 112, 3)
Train batch labels shape: (32,)


In [10]:
def custom_siamese_model(input_shape):
    #inputing first layer of keras model, used Input() from keras.layers
    input1 = Input(input_shape)
    input2 = Input(input_shape)
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))

    # Call the model with the inputs
    embedding1 = model(input1)
    embedding2 = model(input2)

    loss_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    manhattan_distance = loss_layer([embedding1, embedding2])
    output = Dense(1, activation='sigmoid')(manhattan_distance)
    network = Model(inputs=[input1, input2], outputs=output)
    return network

In [11]:
#stop training when val_loss is not improving
early_stopper =  EarlyStopping(monitor='val_loss',min_delta=0,patience=3,verbose=1)
custom_callback = [early_stopper]

In [12]:
model = custom_siamese_model((112,112,3))
#summary of the model
model.summary()

C:\Users\saani\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 128)       │  1,802,016 │ input_layer[0][0… │
│ (Sequential)        │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 128)       │          0 │ sequential[0][0], │
│                     │                   │            │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        129 │ lambda[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,802,145 (6.87 MB)

 Trainable params: 1,802,145 (6.87 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
#1e-4 is 0.0001
optimizer = Adam(learning_rate = 1e-4)
#binary_crossentropy to calculate loss from label and predicted value
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=['accuracy'])


In [14]:
print("Initializing Training !!")
history = model.fit(
    train_gen.datagen(),
    verbose=1,
    steps_per_epoch=len(train_dataset)//32,  # set appropriate steps_per_epoch
    epochs=5,
    validation_data=val_gen.datagen(),
    validation_steps=len(test_dataset)//32,  # set appropriate validation_steps
    callbacks=custom_callback
)


Initializing Training !!


TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.

In [3]:
    model.save('model.h5')

NameError: name 'model' is not defined

In [15]:
history_keys = history.history.keys()

plt.figure(figsize=(12, 4))

# Plot the accuracy curves
if 'accuracy' in history_keys and 'val_accuracy' in history_keys:
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='lower right')

# Plot the loss curves
if 'loss' in history_keys and 'val_loss' in history_keys:
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper right')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

NameError: name 'history' is not defined

In [16]:
# Define the paths to the test images
test_image1_path = 'sign_data/test/050/01_050.png'
test_image2_path = 'sign_data/test/050/04_050.png'

# Load and preprocess the test images
test_image1 = Image.open(test_image1_path)
test_image1 = test_image1.resize((112, 112))
test_image1 = img_to_array(test_image1)
test_image1 = np.expand_dims(test_image1, axis=0)
test_image1 = test_image1.astype('float32')

test_image2 = Image.open(test_image2_path)
test_image2 = test_image2.resize((112, 112))
test_image2 = img_to_array(test_image2)
test_image2 = np.expand_dims(test_image2, axis=0)
test_image2 = test_image2.astype('float32')

# Perform inference on the test images
prediction = model.predict([test_image1, test_image2])

# Print the similarity score
similarity_score = prediction[0][0]
if similarity_score>0.5:
    sign='Forged'
else:
    sign='Genuine'

# Display the test images with the similarity score as legend
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(test_image1[0].astype('uint8'))
plt.title('Test Image 1')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(test_image2[0].astype('uint8'))
plt.title('Test Image 2')
plt.axis('off')

plt.suptitle(f'Similarity Score: {sign}', fontsize=12)
plt.tight_layout()
plt.show()

C:\Users\saani\anaconda3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_1']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step


<IPython.core.display.Javascript object>